In [1]:
import numpy as np
import pandas as pd
from patsy import dmatrices
import warnings

In [2]:
def sigmoid(x):
    '''SIGMOID FUNCTION FOR X'''

    return 1/(1+np.exp(-x))

In [3]:
## Algorith settings

np.random.seed(0) # set the seed
tol=1e-8 # convergence tolerance
lam = None # l2 - regularization
max_iter = 20 # maximum allowed iterations 
r = 0.95 # covariance between x and z
n = 1000 # number of observations (size of dataset to generate)
sigma = 1 # variance of noise - how spread out is the data?

## Model settings

beta_x, beta_z, beta_v = -4, .9, 1 # true beta coefficients
var_x, var_z, var_v = 1, 1, 4 # variances of inputs

## the model specification i want to fit
formula = 'y ~ x + z + v + np.exp(x) + I(v**2 + z)'

In [9]:
# keeping x and z closely related (height and weight)
x, z = np.random.multivariate_normal([0,0], [[var_x, r], [r, var_z]], n).T
# blood pressure
v = np.random.normal(0, var_v, n)**3

# create a pandas dataframe
A = pd.DataFrame({'x': x, 'z': z, 'v': v})

# compute the log offs for our 3 independent variables
# using the sigmoid function
A['log_odds'] = sigmoid(A[['x', 'z', 'v']].dot([beta_x, beta_z, beta_v]) + sigma * np.random.normal(0,1,n))

#compute te probability sample from binomial distribuition
A['y'] = [np.random.binomial(1,p) for p in A.log_odds]

# create a dataframe that encompasses our input data, model formula, and outputs
y, X = dmatrices(formula, A, return_type = 'dataframe')

X.head()

/Users/FabianaBoldrin/PycharmProjects/hackerHank/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.


,Intercept,x,z,v,np.exp(x),I(v ** 2 + z)
0,1.0,-1.611418,-1.566192,15.613483,0.199604,242.214667
1,1.0,-0.080909,0.085959,42.720111,0.922278,1825.093814
2,1.0,0.297531,0.142110,3.530885,1.346531,12.609259
3,1.0,1.412771,1.734809,-57.235945,4.107323,3277.688187
4,1.0,0.204214,-0.302335,-0.074792,1.226561,-0.296741


In [5]:
def catch_singularity(f):
    def silencer(*args, **kwargs):
        try:
            return f(*args, **kwargs)
        except np.linalg.LinAlgError:
            warnings.warn('Algotithm terminated - singular Hessian')
            return args[0]
        return silencer

In [15]:
def newton_step(curr, X, lam=None):
    p = np.array(sigmoid(X.dot(curr[:,0])), ndmin=2).T
    w = np.diag((p*(1-p))[:,0])
    hessian = X.T.dot(w).dot(X)
    grad = X.T.dot(y-p)
    
    if lam:
        step, *_ = np.linalg.lstsq(hessian + lam*np.eye(curr.shape[0]), grad)
    else:
        step, *_ = np.linalg.lstsq(hessian, grad)
    
    beta = curr + step
    
    return beta

In [16]:
def alt_newton_step(curr, X, lam=None):
    p = np.array(sigmoid(X.dot(curr[:,0])), ndmin=2).T
    w = np.diag((p*(1-p))[:,0])
    hessian = X.T.dot(w).dot(X)
    grad = X.T.dot(y-p)
    
    if lam:
        step = np.dot(np.linalg(hessian + lam*np.eye(curr.chape[0])), grad)
    else:
        step = np.dot(np.linalg.inv(hessian), grad)
    
    beta = curr + step
    
    return beta

In [17]:
def check_coefs_convergence(beta_old, beta_new, tol, iters):
    coef_change = np.abs(beta_old - beta_new)
    
    return not (np.any(coef_change > tol) & (iters < max_iter))

In [18]:
## initial conditions
# initial coefficients (weight values), 2 copies, we'll update one
beta_old, beta = np.ones((len(X.columns), 1)), np.zeros((len(X.columns), 1))

iter_count = 0

coefs_converged = False

while not coefs_converged:
    beta_old = beta
    beta = newton_step(beta, X, lam=lam)
    iter_count += 1
    coefs_converged = check_coefs_convergence(beta_old, beta, tol, iter_count)

print('Iterations: {}'.format(iter_count))
print('Beta: {}'.format(beta))

/Users/FabianaBoldrin/PycharmProjects/hackerHank/venv/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  # Remove the CWD from sys.path while we load stuff.
/Users/FabianaBoldrin/PycharmProjects/hackerHank/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.


Iterations: 19
Beta: [[-5.54864946e+29]
 [ 3.94495505e+29]
 [-5.60359599e+29]
 [ 7.77400045e+29]
 [-4.79447792e+27]
 [ 7.99669363e+29]]
